In [25]:
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
from datetime import date, time, datetime
from operator import attrgetter

In [26]:
data = pd.read_excel('ecom_data.xlsx', sheet_name=['Orders','Customers','Sales_people','Returns'])

c:\Users\Bogdan\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\Bogdan\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\Bogdan\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [27]:
orders = data['Orders']
customers = data['Customers']
sales_people = data['Sales_people']
returns = data['Returns']

In [28]:
returns.head()

,Returned,Order ID
0,Yes,CA-2017-100762
1,Yes,CA-2017-100762
2,Yes,CA-2017-100762
3,Yes,CA-2017-100762
4,Yes,CA-2017-100867


In [29]:
# merge orders with customers on left
orders_customers = pd.merge(orders, customers, on='Customer ID', how='left')
orders_customers.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State,Postal Code,Region,...,Discount,Profit,Segment,Country,CityOfLiving,Phone,Email_falg,App,App_push,website_registration_date
0,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,0.00,41.9136,Consumer,United States,Henderson,1,1,1,0,2018-10-20
1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,0.00,219.5820,Consumer,United States,Henderson,1,1,1,0,2018-10-20
2,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,United States,Los Angeles,California,90036.0,West,...,0.00,6.8714,Corporate,United States,Los Angeles,1,1,1,1,2019-06-16
3,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,0.45,-383.0310,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30
4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,0.20,2.5164,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30


In [30]:
# merge (orders with customers) with returns on left

orders_customers_returns = pd.merge(orders_customers, returns, on='Order ID', how='left')
orders_customers_returns.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State,Postal Code,Region,...,Profit,Segment,Country,CityOfLiving,Phone,Email_falg,App,App_push,website_registration_date,Returned
0,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,41.9136,Consumer,United States,Henderson,1,1,1,0,2018-10-20,NaN
1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,219.5820,Consumer,United States,Henderson,1,1,1,0,2018-10-20,NaN
2,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,United States,Los Angeles,California,90036.0,West,...,6.8714,Corporate,United States,Los Angeles,1,1,1,1,2019-06-16,NaN
3,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,-383.0310,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30,NaN
4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,2.5164,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30,NaN


In [31]:
# make final data => merge (orders with customers with returns) with sales_people on left
final_data = pd.merge(orders_customers_returns, sales_people, on='Region', how='left')
final_data.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State,Postal Code,Region,...,Segment,Country,CityOfLiving,Phone,Email_falg,App,App_push,website_registration_date,Returned,Person
0,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,Consumer,United States,Henderson,1,1,1,0,2018-10-20,NaN,Cassandra Brandow
1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,Consumer,United States,Henderson,1,1,1,0,2018-10-20,NaN,Cassandra Brandow
2,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,United States,Los Angeles,California,90036.0,West,...,Corporate,United States,Los Angeles,1,1,1,1,2019-06-16,NaN,Anna Andreadi
3,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30,NaN,Cassandra Brandow
4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30,NaN,Cassandra Brandow


In [19]:
# see NaNs in column Returned -> have to fillna with (No) cause in table returns had info only about orders which already returns (Yes)
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12420 entries, 0 to 12419
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Order ID                   12420 non-null  object        
 1   Order Date                 12420 non-null  datetime64[ns]
 2   Ship Date                  12420 non-null  datetime64[ns]
 3   Ship Mode                  12420 non-null  object        
 4   Customer ID                12420 non-null  object        
 5   Country/Region             12420 non-null  object        
 6   City                       12420 non-null  object        
 7   State                      12420 non-null  object        
 8   Postal Code                12409 non-null  float64       
 9   Region                     12420 non-null  object        
 10  Product ID                 12420 non-null  object        
 11  Category                   12420 non-null  object        
 12  Sub-

In [32]:
#fillna with No
final_data['Returned'] = final_data['Returned'].fillna('No')

In [22]:
final_data['Returned'].value_counts()

Returned
No     9194
Yes    3226
Name: count, dtype: int64

In [23]:
final_data.describe()

,Order Date,Ship Date,Postal Code,Sales,Quantity,Discount,Profit,Phone,Email_falg,App,App_push,website_registration_date
count,12420,12420,12409.000000,12420.000000,12420.000000,12420.000000,12420.000000,12420.0,12420.000000,12420.000000,12420.000000,12420
mean,2019-05-13 14:10:46.956521728,2019-05-17 12:07:39.130434816,57417.516480,233.629369,3.785024,0.154403,28.995935,1.0,0.898470,0.590741,0.312560,2017-08-23 07:30:46.956521728
min,2017-01-03 00:00:00,2017-01-07 00:00:00,1040.000000,0.444000,1.000000,0.000000,-6599.978000,1.0,0.000000,0.000000,0.000000,2016-10-12 00:00:00
25%,2018-05-28 00:00:00,2018-06-01 00:00:00,24153.000000,17.904000,2.000000,0.000000,1.969800,1.0,1.000000,0.000000,0.000000,2017-03-19 00:00:00
50%,2019-07-18 00:00:00,2019-07-23 12:00:00,60623.000000,55.392000,3.000000,0.150000,9.072000,1.0,1.000000,1.000000,0.000000,2017-07-13 00:00:00
75%,2020-06-08 00:00:00,2020-06-10 00:00:00,90045.000000,212.680000,5.000000,0.200000,29.372000,1.0,1.000000,1.000000,1.000000,2017-11-09 00:00:00
max,2020-12-30 00:00:00,2021-01-05 00:00:00,99301.000000,22638.480000,14.000000,0.800000,8399.976000,1.0,1.000000,1.000000,1.000000,2020-10-11 00:00:00
std,NaN,NaN,32749.815711,620.762848,2.231328,0.206124,246.506696,0.0,0.302041,0.491717,0.463555,NaN


In [24]:
# drop 11 lines where no info about postal code
final_data.dropna()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State,Postal Code,Region,...,Segment,Country,CityOfLiving,Phone,Email_falg,App,App_push,website_registration_date,Returned,Person
0,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,Consumer,United States,Henderson,1,1,1,0,2018-10-20,No,Cassandra Brandow
1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,Consumer,United States,Henderson,1,1,1,0,2018-10-20,No,Cassandra Brandow
2,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,United States,Los Angeles,California,90036.0,West,...,Corporate,United States,Los Angeles,1,1,1,1,2019-06-16,No,Anna Andreadi
3,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30,No,Cassandra Brandow
4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30,No,Cassandra Brandow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12415,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,United States,Costa Mesa,California,92627.0,West,...,Consumer,United States,Seattle,1,1,0,0,2017-01-23,Yes,Anna Andreadi
12416,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,United States,Costa Mesa,California,92627.0,West,...,Consumer,United States,Seattle,1,1,0,0,2017-01-23,Yes,Anna Andreadi
12417,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,United States,Costa Mesa,California,92627.0,West,...,Consumer,United States,Seattle,1,1,0,0,2017-01-23,Yes,Anna Andreadi
12418,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,United States,Costa Mesa,California,92627.0,West,...,Consumer,United States,Seattle,1,1,0,0,2017-01-23,Yes,Anna Andreadi


In [33]:
final_data.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State,Postal Code,Region,...,Segment,Country,CityOfLiving,Phone,Email_falg,App,App_push,website_registration_date,Returned,Person
0,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,Consumer,United States,Henderson,1,1,1,0,2018-10-20,No,Cassandra Brandow
1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,United States,Henderson,Kentucky,42420.0,South,...,Consumer,United States,Henderson,1,1,1,0,2018-10-20,No,Cassandra Brandow
2,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,United States,Los Angeles,California,90036.0,West,...,Corporate,United States,Los Angeles,1,1,1,1,2019-06-16,No,Anna Andreadi
3,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30,No,Cassandra Brandow
4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,United States,Fort Lauderdale,Florida,33311.0,South,...,Consumer,United States,Fort Lauderdale,1,1,0,0,2018-09-30,No,Cassandra Brandow
